### 임베딩 Word2Vec

In [1]:
!pip install konlpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 2.5 MB/s eta 0:00:00


In [2]:
!pip install python-mecab-ko

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 573.9/573.9 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 18.0 MB/s eta 0:00:00


In [3]:
!pip install gensim

In [4]:
import pandas as pd
import numpy as np

from konlpy.tag import Okt
from mecab import MeCab as Mecab
from gensim.models.word2vec import Word2Vec

In [5]:
file_path = '/content/drive/MyDrive/KDT/딥러닝/자연어처리/nsmc.txt'

#df = pd.read_table(file_path)
df = pd.read_csv(file_path, sep = '\t')
df

,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1
...,...,...,...
199995,8963373,포켓 몬스터 짜가 ㅡㅡ;;,0
199996,3302770,쓰.레.기,0
199997,5458175,완전 사이코영화. 마지막은 더욱더 이 영화의질을 떨어트린다.,0
199998,6908648,왜난 재미없었지 ㅠㅠ 라따뚜이 보고나서 스머프 봐서 그런가 ㅋㅋ,0


In [8]:
df.isnull().sum()

id          0
document    8
label       0
dtype: int64

In [6]:
cleaned_df = df.dropna()

In [7]:
#cleaned_df = cleaned_df.reset_index(drop = True)
#cleaned_df

### reset_index --> 인덱스 재설정
cleaned_df.reset_index(drop=True, inplace=True)

# 결과 확인하기
print(cleaned_df)

              id                                           document  label
0        8112052                                어릴때보고 지금다시봐도 재밌어요ㅋㅋ      1
1        8132799  디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...      1
2        4655635               폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.      1
3        9251303  와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...      1
4       10067386                        안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.      1
...          ...                                                ...    ...
199987   8963373                                     포켓 몬스터 짜가 ㅡㅡ;;      0
199988   3302770                                              쓰.레.기      0
199989   5458175                  완전 사이코영화. 마지막은 더욱더 이 영화의질을 떨어트린다.      0
199990   6908648                왜난 재미없었지 ㅠㅠ 라따뚜이 보고나서 스머프 봐서 그런가 ㅋㅋ      0
199991   8548411                                    포풍저그가나가신다영차영차영차      0

[199992 rows x 3 columns]


In [ ]:
import re
'''
pattern = '[^가-힣 ]'
repl = ''  # 길이가 0인 문자열 --> 제거
string = document 칼럼의 모든 review
'''
'''
def remove(x):
  return re.sub(pattern, '', x)
'''

cleaned_df.loc[:, 'document'].apply(lambda x : re.sub('[^가-힣 ]', '', x))

In [ ]:
### pd.series.str.replace() 함수 --> 한글과 공백 외 문자 제거
pattern = '[^가-힣 ]'
cleaned_df.loc[:, 'document'] = cleaned_df.loc[:, 'document'].str.replace(pat = pattern, repl = "", regex = True)

# 결과 확인하기
print(cleaned_df)

              id                                           document  label
0        8112052                                  어릴때보고 지금다시봐도 재밌어요      1
1        8132799  디자인을 배우는 학생으로 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산업...      1
2        4655635                   폴리스스토리 시리즈는 부터 뉴까지 버릴께 하나도 없음 최고      1
3        9251303   와 연기가 진짜 개쩔구나 지루할거라고 생각했는데 몰입해서 봤다 그래 이런게 진짜 영화지      1
4       10067386                         안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화      1
...          ...                                                ...    ...
199987   8963373                                         포켓 몬스터 짜가       0
199988   3302770                                                쓰레기      0
199989   5458175                    완전 사이코영화 마지막은 더욱더 이 영화의질을 떨어트린다      0
199990   6908648                    왜난 재미없었지  라따뚜이 보고나서 스머프 봐서 그런가       0
199991   8548411                                    포풍저그가나가신다영차영차영차      0

[199992 rows x 3 columns]


In [ ]:
### 불용어 정의
stopwords = ['의', '가', '이', '은', '을', '것', '때', '그', '들', '는', '좀', '잘', '걍', '과', '도', '를', '자','에', '와', '한', '으로', '부터', '까지']

#### Okt 형태소 분석기를 이용한 Word2Vec 실습

In [ ]:
### Okt 형태소 분석기 --> 토큰화 작업 + 불용어 제거

# Okt 형태소 분석기 객체 생성
okt = Okt()

# 형태소 분석
result = []
for review in cleaned_df.loc[:, 'document']:
  print(f'review : \n{review}')
  print('-'*80)

  # review 1개 --> 토큰화 + 불용어 제거 결과 저장
  review_tokenized = []

  # 형태소 단위 토큰화
  tokenized_review = okt.morphs(review)
  print(f'형태소 분석의 결과 : \n{tokenized_review}')
  print('-'*80)

  # 불용어 제거
  for morph in tokenized_review:
    if morph not in stopwords:
      review_tokenized.append(morph)
  print(f'리뷰 1개를 처리한 결과 : \n{review_tokenized}')
  print('-'*80)
  result.append(review_tokenized)

print(f'최종 결과 확인 : \n{result}')

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
['평점', '조', '지기']
--------------------------------------------------------------------------------
리뷰 1개를 처리한 결과 : 
['평점', '조', '지기']
--------------------------------------------------------------------------------
review : 
점도 주기 아깝 보면볼수록 짜증나는 거짓말로채우는 드라마 꼴보기도싫어 주간을 안보고 다시 보려했더니 아직도 거짓말뿐저녁시간에 가족들 함께보는시간에 거짓말과 못된짓만 보여주는 회 회가 짜증만 돋구는군요 오늘도 역시나 채널돌리게 되네요
--------------------------------------------------------------------------------
형태소 분석의 결과 : 
['점도', '주기', '아깝', '보면', '볼수록', '짜증나는', '거짓말', '로', '채우는', '드라마', '꼴', '보기', '도', '싫어', '주간', '을', '안보', '고', '다시', '보려', '했더니', '아직도', '거짓말', '뿐', '저녁', '시간', '에', '가족', '들', '함께', '보는', '시간', '에', '거짓말', '과', '못', '된', '짓', '만', '보여주는', '회', '회', '가', '짜증', '만', '돋구는군요', '오늘', '도', '역시', '나', '채널', '돌리게', '되네요']
--------------------------------------------------------------------------------
리뷰 1개를 처리한 결과 : 
['점도', '주기', '아깝', '보면', '볼수록', '짜증나는', '거짓말', '로', '채우는', '드라마', '꼴', '보기', '싫어', '주간', '안보', '고', 

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



##### 모델 저장 및 불러오기

In [ ]:
### CBOW, skip-gram 모델 --> 단어 사전(min_count=k 만족) + 임베딩 벡터 생성

# CBOW 모델 생성 + 실행
cbow_okt = Word2Vec(sentences = result, vector_size=100, window = 5, min_count=5, workers = 1, sg=0)

# skip-gram 모델 생성 + 실행
sg_okt = Word2Vec(sentences=result, vector_size=100, window=5, min_count=5, workers=1, sg=1)

In [ ]:
### CBOW, skip-gram 모델 저장하기 --> model.save()

# 저장 경로 설정
cbow_okt_path = '/content/drive/MyDrive/KDT/딥러닝/자연어처리/cbow_okt.model'
sg_okt_path = '/content/drive/MyDrive/KDT/딥러닝/자연어처리/sg_okt.model'

cbow_okt.save(cbow_okt_path)

sg_okt.save(sg_okt_path)

In [ ]:
### CBOW, skip-gram 모델 불러오기

# CBOW 모델 불러오기
loaded_cbow_okt = Word2Vec.load(cbow_okt_path)

# skip-gram 모델 불러오기
loaded_sg_okt = Word2Vec.load(sg_okt_path)

##### CBOW 모델 확인

In [ ]:
### COBW 모델 --> 생성된 단어 사전과 임베딩 벡터 확인
'''
1. 단어 사전, 임베딩 벡터 저장 --> model.wv
2. 단어 사전 : model.wv.key_to_index
3. model.wv['word']를 통해 특정 단어의 embedding vector를 확인할 수 있다
4. model.wv.vectors 속성을 이용하여 생성된 단어 사전의 전체 value(embedding vector)를 확인할 수 있다.
5. model.wv.similarity(w1=w1, w2=w2) 메서드를 이용하여 단어 간의 유사도를 비교할 수 있다.
6. model.wv.most_similar(word, topn=n) 메서드를 이용하여 특정 단어와 가장 가까운 단어
n개를 추출할 수 있다.
'''

"\n1. 단어 사전, 임베딩 벡터 저장 --> model.wv\n2. 단어 사전 : model.wv.key_to_index\n3. model.wv['word']를 통해 특정 단어의 embedding vector를 확인할 수 있다\n4. model.wv.vectors 속성을 이용하여 생성된 단어 사전의 전체 value(embedding vector)를 확인할 수 있다.\n5. model.wv.similarity(w1=w1, w2=w2) 메서드를 이용하여 단어 간의 유사도를 비교할 수 있다.\n6. model.wv.most_similar(word, topn=n) 메서드를 이용하여 특정 단어와 가장 가까운 단어 \nn개를 추출할 수 있다.\n"

In [ ]:
# 단어 사전 확인하기 1,2
vocab_cbow_okt = loaded_cbow_okt.wv.key_to_index
print(f'단어 사전의 자료형 : {type(vocab_cbow_okt)}')
print('-'*80)
print(f'단어 사전 확인 : \n{vocab_cbow_okt}')

# 특정 단어 3
word_vector1 = loaded_cbow_okt.wv['최민식']
print(f'최민식이라는 단어의 임베딩 벡터 : \n{word_vector1}')
print('-'*80)

word_vector2 = loaded_cbow_okt.wv['유해진']
print(f'유해진이라는 단어의 임베딩 벡터 : \n{word_vector2}')
print('-'*80)

# 두 단어의 유사도 비교 5
sim1 = loaded_cbow_okt.wv.similarity('최민식', '유해진')
print(sim1)
print('-'*80)

# 특정 단어와 가까운 단어 n개 추출 6
top5_1 = loaded_cbow_okt.wv.most_similar('최민식', topn = 5)
print(top5_1)

단어 사전의 자료형 : <class 'dict'>
--------------------------------------------------------------------------------
단어 사전 확인 : 
{'영화': 0, '너무': 1, '다': 2, '정말': 3, '만': 4, '적': 5, '진짜': 6, '로': 7, '점': 8, '에서': 9, '평점': 10, '연기': 11, '최고': 12, '내': 13, '나': 14, '인': 15, '안': 16, '이런': 17, '생각': 18, '못': 19, '스토리': 20, '왜': 21, '드라마': 22, '게': 23, '이다': 24, '감동': 25, '사람': 26, '보고': 27, '하는': 28, '하고': 29, '고': 30, '말': 31, '더': 32, '배우': 33, '감독': 34, '아': 35, '그냥': 36, '거': 37, '요': 38, '본': 39, '재미': 40, '시간': 41, '내용': 42, '중': 43, '뭐': 44, '보다': 45, '없는': 46, '쓰레기': 47, '네': 48, '수': 49, '지': 50, '봤는데': 51, '사랑': 52, '작품': 53, '볼': 54, '할': 55, '다시': 56, '하나': 57, '없다': 58, '마지막': 59, '좋은': 60, '이건': 61, '저': 62, '정도': 63, '같은': 64, '완전': 65, '있는': 66, '입니다': 67, '처음': 68, '장면': 69, '액션': 70, '주인공': 71, '이렇게': 72, '걸': 73, '보는': 74, '최악': 75, '하': 76, '개': 77, '이야기': 78, '지금': 79, '돈': 80, '별로': 81, '봐도': 82, '임': 83, '참': 84, '연출': 85, '느낌': 86, '끝': 87, '없고': 88, '라': 89, '인데': 90, '듯':

##### skip-gram(sg) 모델 확인

In [ ]:
### sg(skip-gram) 모델 --> --> 생성된 단어 사전과 임베딩 벡터 확인

# 단어 사전 확인하기 1,2
vocab_sg_okt = loaded_sg_okt.wv.key_to_index
print(f'단어 사전의 자료형 : {type(vocab_sg_okt)}')
print('-'*80)
print(f'단어 사전 확인 : \n{vocab_sg_okt}')

# 특정 단어 3
word_vector1 = loaded_sg_okt.wv['최민식']
print(f'최민식이라는 단어의 임베딩 벡터 : \n{word_vector1}')
print('-'*80)

word_vector2 = loaded_sg_okt.wv['유해진']
print(f'유해진이라는 단어의 임베딩 벡터 : \n{word_vector2}')
print('-'*80)

# 두 단어의 유사도 비교 5
sim2 = loaded_sg_okt.wv.similarity('최민식', '유해진')
print(sim2)
print('-'*80)

# 특정 단어와 가까운 단어 n개 추출 6
top5_2 = loaded_sg_okt.wv.most_similar('최민식', topn = 5)
print(top5_2)

단어 사전의 자료형 : <class 'dict'>
--------------------------------------------------------------------------------
단어 사전 확인 : 
{'영화': 0, '너무': 1, '다': 2, '정말': 3, '만': 4, '적': 5, '진짜': 6, '로': 7, '점': 8, '에서': 9, '평점': 10, '연기': 11, '최고': 12, '내': 13, '나': 14, '인': 15, '안': 16, '이런': 17, '생각': 18, '못': 19, '스토리': 20, '왜': 21, '드라마': 22, '게': 23, '이다': 24, '감동': 25, '사람': 26, '보고': 27, '하는': 28, '하고': 29, '고': 30, '말': 31, '더': 32, '배우': 33, '감독': 34, '아': 35, '그냥': 36, '거': 37, '요': 38, '본': 39, '재미': 40, '시간': 41, '내용': 42, '중': 43, '뭐': 44, '보다': 45, '없는': 46, '쓰레기': 47, '네': 48, '수': 49, '지': 50, '봤는데': 51, '사랑': 52, '작품': 53, '볼': 54, '할': 55, '다시': 56, '하나': 57, '없다': 58, '마지막': 59, '좋은': 60, '이건': 61, '저': 62, '정도': 63, '같은': 64, '완전': 65, '있는': 66, '입니다': 67, '처음': 68, '장면': 69, '액션': 70, '주인공': 71, '이렇게': 72, '걸': 73, '보는': 74, '최악': 75, '하': 76, '개': 77, '이야기': 78, '지금': 79, '돈': 80, '별로': 81, '봐도': 82, '임': 83, '참': 84, '연출': 85, '느낌': 86, '끝': 87, '없고': 88, '라': 89, '인데': 90, '듯':

#### Mecab 형태소 분석기를 이용한 Word2Vec 실습

In [ ]:
### Mecab 형태소 분석기 --> 토큰화 + 불용어 제거 --> 입력 데이터 생성

mecab = Mecab()

result2 = []

for review in cleaned_df.loc[:, 'document']:
  review_tokenized = []

  tokenized_review = mecab.morphs(review)

  for morph in tokenized_review:
    if morph not in stopwords:
      review_tokenized.append(morph)

  result2.append(review_tokenized)

print(result2)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



##### 모델 저장 및 불러오기

In [ ]:
cbow_mecab = Word2Vec(sentences = result2, vector_size = 100, window = 5, min_count = 5, workers = 1, sg = 0)

sg_mecab = Word2Vec(sentences = result2, vector_size = 100, window = 5, min_count = 5, workers = 1, sg = 1)

In [ ]:
cbow_mecab_path = '/content/drive/MyDrive/KDT/딥러닝/자연어처리/cbow_mecab.model'
sg_mecab_path = '/content/drive/MyDrive/KDT/딥러닝/자연어처리/sg_mecab.model'

cbow_mecab.save(cbow_mecab_path)
sg_mecab.save(sg_mecab_path)

In [ ]:
loaded_cbow_mecab = Word2Vec.load(cbow_mecab_path)
loaded_sg_mecab = Word2Vec.load(sg_mecab_path)

##### CBOW 모델 확인

In [ ]:
vocab_cbow_mecab = loaded_cbow_mecab.wv.key_to_index
print(type(vocab_cbow_mecab))
print('-'*80)
print(vocab_cbow_mecab)
print('-'*80)

word_vector1 = loaded_cbow_mecab.wv['최민식']
print(word_vector1)
print('-'*80)
word_vector2 = loaded_cbow_mecab.wv['유해진']
print(word_vector2)
print('-'*80)

sim1 = loaded_cbow_mecab.wv.similarity('최민식', '유해진')
print(sim1)
print('-'*80)

top5_1 = loaded_cbow_mecab.wv.most_similar('최민식', topn = 5)
print(top5_1)

<class 'dict'>
--------------------------------------------------------------------------------
{'영화': 0, '다': 1, '고': 2, '하': 3, '보': 4, '게': 5, '지': 6, '있': 7, '없': 8, '좋': 9, '나': 10, '었': 11, '만': 12, '는데': 13, '너무': 14, '봤': 15, '안': 16, '적': 17, '정말': 18, '로': 19, '음': 20, '재밌': 21, '아': 22, '네요': 23, '어': 24, '지만': 25, '같': 26, '진짜': 27, '에서': 28, '했': 29, '기': 30, '네': 31, '점': 32, '않': 33, '거': 34, '았': 35, '수': 36, '되': 37, '면': 38, '인': 39, '연기': 40, '말': 41, '최고': 42, '주': 43, '평점': 44, '내': 45, '이런': 46, '던': 47, '어요': 48, '왜': 49, '할': 50, '겠': 51, '해': 52, '습니다': 53, '스토리': 54, '듯': 55, '아니': 56, '생각': 57, '더': 58, '드라마': 59, '싶': 60, '사람': 61, '감동': 62, '배우': 63, '함': 64, '본': 65, '볼': 66, '보다': 67, '내용': 68, '뭐': 69, '만들': 70, '알': 71, '감독': 72, '라': 73, '재미': 74, '그냥': 75, '시간': 76, '재미있': 77, '중': 78, '지루': 79, '잼': 80, '였': 81, '재미없': 82, '년': 83, '사랑': 84, '냐': 85, '쓰레기': 86, '못': 87, '서': 88, '라고': 89, '니': 90, '번': 91, '다시': 92, '면서': 93, '나오': 94, '하나': 95, '작품'

##### skip-gram(sg) 모델 확인

In [ ]:
vocab_sg_mecab = loaded_sg_mecab.wv.key_to_index
print(type(vocab_sg_mecab))
print('-'*80)
print(vocab_sg_mecab)
print('-'*80)

word_vector1 = loaded_sg_mecab.wv['최민식']
print(word_vector1)
print('-'*80)
word_vector2 = loaded_sg_mecab.wv['유해진']
print(word_vector2)
print('-'*80)

sim2 = loaded_sg_mecab.wv.similarity('최민식', '유해진')
print(sim2)
print('-'*80)

top5_2 = loaded_sg_mecab.wv.most_similar('최민식', topn = 5)
print(top5_2)

<class 'dict'>
--------------------------------------------------------------------------------
{'영화': 0, '다': 1, '고': 2, '하': 3, '보': 4, '게': 5, '지': 6, '있': 7, '없': 8, '좋': 9, '나': 10, '었': 11, '만': 12, '는데': 13, '너무': 14, '봤': 15, '안': 16, '적': 17, '정말': 18, '로': 19, '음': 20, '재밌': 21, '아': 22, '네요': 23, '어': 24, '지만': 25, '같': 26, '진짜': 27, '에서': 28, '했': 29, '기': 30, '네': 31, '점': 32, '않': 33, '거': 34, '았': 35, '수': 36, '되': 37, '면': 38, '인': 39, '연기': 40, '말': 41, '최고': 42, '주': 43, '평점': 44, '내': 45, '이런': 46, '던': 47, '어요': 48, '왜': 49, '할': 50, '겠': 51, '해': 52, '습니다': 53, '스토리': 54, '듯': 55, '아니': 56, '생각': 57, '더': 58, '드라마': 59, '싶': 60, '사람': 61, '감동': 62, '배우': 63, '함': 64, '본': 65, '볼': 66, '보다': 67, '내용': 68, '뭐': 69, '만들': 70, '알': 71, '감독': 72, '라': 73, '재미': 74, '그냥': 75, '시간': 76, '재미있': 77, '중': 78, '지루': 79, '잼': 80, '였': 81, '재미없': 82, '년': 83, '사랑': 84, '냐': 85, '쓰레기': 86, '못': 87, '서': 88, '라고': 89, '니': 90, '번': 91, '다시': 92, '면서': 93, '나오': 94, '하나': 95, '작품'